### 🧊 Lab 4 — Value Iteration and Policy Iteration for Frozen Lake
In this lab we will continue our exploration of Markov Decision Processes (MDPs) 
using the simplified **FrozenLake** environment. Building on Lab 3, we will 
practice two key ideas:

1. **Finding the Optimal Policy (Value Iteration)**  We apply value iteration to compute the optimal state values and extract the corresponding optimal policy that maximizes long-term return.

2. **Finding the Optimal Policy (Policy Iteration)** We apply policy iteration, which alternates between evaluating the current policy and improving it until convergence to the optimal policy.

In [83]:
import gymnasium as gym
import numpy as np
np.set_printoptions(precision=2, suppress=True)
import random
from gymnasium.envs.toy_text.frozen_lake import generate_random_map

In [84]:
# Define a smaller 3x3 map
DESC_3x3 = [
    "SFF",
    "FHF",
    "FFG",
]
env = gym.make("FrozenLake-v1",desc=DESC_3x3,is_slippery=True, render_mode="ansi")
obs, info = env.reset(seed=42)
#print(env.render())

In [85]:
P_LEFT = [
    [2, 1, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 1, 0, 0],
    [0, 1, 0, 0, 3, 1, 0, 1, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 1, 0, 0, 2, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 1, 0, 0, 1],
]

P_DOWN = [
    [1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 3, 1, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 2, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 1, 0, 1, 2],
]

P_RIGHT = [
    [1, 0, 0, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 2, 0, 0, 1, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 1, 0, 1, 3, 0, 0, 1, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 2],
]

P_UP = [
    [2, 1, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 1, 0, 0],
    [0, 1, 0, 0, 3, 1, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 2, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 3],
]
P_all = np.array([P_LEFT, P_DOWN, P_RIGHT, P_UP]).transpose(0, 2, 1)/3
POLICY = [
    1,  # state 0 (top-left)
    2,  # state 1
    1,  # state 2
    1,  # state 3
    1,  # state 4 (center, possibly hole)
    1,  # state 5
    2,  # state 6
    2,  # state 7
    2,  # state 8 (goal state)
]

n_states = len(POLICY)
P_pi = np.zeros((n_states, n_states))
for s in range(n_states):
    a = POLICY[s]             # action chosen at state s
    P_pi[s, :] = P_all[a, s]  # copy the probabilities for that action

Reward = [
0,  # state 0 (top-left)
0,  # state 1
0,  # state 2
0,  # state 3
0,  # state 4 (center, possibly hole)
0,  # state 5
0,  # state 6
0,  # state 7
1,  # state 8 (goal state)
]
Reward = np.array(Reward)
gamma = 0.9

## Part 1: Finding the Optimal Policy (Value Iteration)

So far, we have focused on **policy evaluation** — estimating the value of a 
given policy. In this part, we move to **control**, where the goal is to find 
the **optimal policy** that maximizes long-term returns.

The key idea is to apply **value iteration**, which combines evaluation and 
improvement into a single update rule:

$$
V_{k+1}(s) \;=\; \max_a \Big[ R(s) + \gamma \sum_{s'} P(s'|s,a) \, V_k(s') \Big].
$$

Once the value function converges, we extract the optimal policy by choosing, 
in each state, the action that achieves the maximum:

$$
\pi^*(s) = \arg\max_a \Big[ R(s) + \gamma \sum_{s'} P(s'|s,a) V^*(s') \Big].
$$

### Steps
1. Initialize the value function $V_0(s)$ arbitrarily (e.g., all zeros).  
2. Repeatedly update each state’s value using the **max over actions** rule.  
3. Stop when values converge (the updates change very little).  
4. Derive the optimal policy $\pi^*$ by picking the greedy action at each state.  

### What to do
- Implement value iteration with your transition model `P_all_prob` and rewards.  
- Print the **optimal state values**.  
- Print the **optimal policy** as action indices (0 = LEFT, 1 = DOWN, 2 = RIGHT, 3 = UP).  


In [86]:
# Your time to work on it

## Part 2: Finding the Optimal Policy (Truncated Policy Iteration)
So far, we have seen two extremes:  
- **Value Iteration**, which maximizes at every update, and  
- **Policy Iteration**, which fully evaluates a policy before improving it.  

**Truncated (Modified) Policy Iteration** strikes a balance by performing only a **few** policy-evaluation sweeps before each policy-improvement step.

Formally, we alternate between **partial policy evaluation** and **policy improvement**:

$$
V^{(j+1)}(s) \;=\; R(s) \;+\; \gamma \sum_{s'} P(s' \mid s, \pi_k(s))\, V^{(j)}(s'),
\quad j = 0,1,\dots,m-1,
$$

followed by:

$$
\pi_{k+1}(s) \;=\; \arg\max_{a} \Big[ \, R(s) \;+\; \gamma \sum_{s'} P(s' \mid s, a)\, V^{(m)}(s') \, \Big].
$$

### Steps
1. Initialize the value function $ V_0(s) $ (e.g., all zeros) and an initial policy $ \pi_0 $.  
2. **Partial Evaluation**: With $ \pi_k$ fixed, perform only $ m $ Bellman **expectation** sweeps to update $ V $.  
3. **Policy Improvement**: Update the policy by choosing the action that maximizes the expected return with the updated $V $.  
4. Repeat Steps 2–3 until the policy no longer changes (or value/policy changes are below a small threshold).

### What to do
- Implement truncated policy iteration using your transition model `P_all_prob` and rewards `R`.  
- Choose a small number of evaluation sweeps \( m \) (e.g., 1–3) **or** use a tolerance \( \theta \) to stop early within each evaluation phase.  
- Print the **final state values** and the **resulting policy** as action indices  
  `(0 = LEFT, 1 = DOWN, 2 = RIGHT, 3 = UP)`.  
- (Optional) Compare iterations/time vs. **full policy iteration** and **value iteration** on your map.


In [87]:
# Your time to work on it

## Comparison: Reproduce the Convergence Plot (PI vs. VI vs. Truncated PI)

Goal: make a single figure showing how **Value Iteration (VI)**, **Policy Iteration (PI)**, and
**Truncated Policy Iteration (TPI)** converge toward the **optimal state value** \(V^\*\) for the
same MDP (e.g., your FrozenLake map).

### Setup
- Use the **same environment**, discount factor $\gamma$, and reward convention as in Parts 1–4.
- Fix a **representative state** $s_\text{ref}$ (recommended: the start state `S`).  
  The y-axis will be $V_k(s_\text{ref})$.
- Use **synchronous** updates for VI/TPI to keep curves smooth (one full sweep = 1 iteration).

### Plotting instructions
- x-axis: iteration index $k$.
- y-axis: $V_k(s_\text{ref})$.
- Draw:
  - Magenta line with diamond markers for **Value Iteration** (`vals_vi`).
  - Blue line with circular markers for **Policy Iteration** (`vals_pi`).
  - Black line with square markers for **Truncated PI** (`vals_tpi`).
  - Red **horizontal** line at $y = V^*(s_\text{ref})$ labeled **Optimal state value**.
- Add legend:
  - Policy iteration
  - Value iteration
  - Truncated policy iteration
  - Optimal state value
- Label axes: `k` (iterations) and `V` (state value).
- Add light grid lines for readability.

In [ ]:
# Your time to work on it